---
title: "2023_Kronensicherung_Plesse_001_Versuchsablauf"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_Kraefte_Schwingungen

## Baumdaten, Gerädeanordung und Versuchsaufzeichung

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment) und installiere die Pakete linescale3 (LS3) und treeqinetic (PTQ) inklusive kj_core und kj_logger und weiteren requirements.

## Arbeitsumgebung vorbereiten

Es werden zuerst benötigte Standard-Pakete importiert. Nachfolgend die zwei extra geschriebenen Pakete LS3 und PTQ. Fehler beim Import dieser zwei Pakete sind ggf. Bugs. Beide Pakete nutzen eine gemeinsame CodeBasis in den Paketen kj_core (Core-Package) und kj_logger (individualisiertes Logging des Verarbeitungs-Prozesses). Diese sollte i. d. R. über die requirements mit installiert werden.

### IMPORT: Importieren von Standardbibliotheken

Die folgenden Bibliotheken werden importiert, um grundlegende Funktionen für Strukturierung, Datenverarbeitung, Plotting und statistische Auswertung bereit zu stellen.

In [1]:
# Struktur
from pathlib import Path
from typing import Dict, List

# Datenverarbeitung
import json
from IPython.display import Markdown, display
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from slugify import slugify  # Slugify ums strings in standard Formate zu überführen

Lade allgemeine Export-Funktionen, um die Daten als Latex-Tabellen zu exportieren

In [2]:
from kj_core.utils.latex_export import (
    save_latex_table,
    build_data_dict_df
)

### IMPORT: Importieren von eigenen Modulen

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur), ausgelagert in gemeinsame Config für verschiedene Notebooks

In [3]:
# Importiere alle Einstellungen aus der project_config.py
from project_config import (
    analyse_name,
    data_path,
    working_directory,
    data_export_directory,
    latex_export_directory
)



## Vorbemerkung

Die Daten wurden während des Versuchs als GoogleSheet erfasst und abschließend als CSV-Dateien exportiert, um ein einfaches Einlesen zur ermöglichen.
Nachfolgend werden Datentypen für alle Spalten explizit in den *_data_dict.json angegeben, da es teils durch Python-Pandas zu Fehlerkennungen kommt.
Zusätzlich werden teils Spaltennamen vereinheitlicht oder benannt.

## Daten des Versuchsbaums (tree_df)

### IMPORT: Daten und Datendokumentation

Die `tree.csv` liefert spezifische Informationen über den Versuchsbaum. Da nur ein Versuchsbaum, sind hier Daten und Datendokumentation in einer Datei.

In [4]:
tree_df = pd.read_csv(
    data_path / 'tree.csv',
    sep=';', decimal=',',
    na_values='NA',
    dtype='string'  # alle Spalten als string
)

### ANALYSE: Explorative Datenanalyse

In [5]:
#tree_df

In [6]:
# In Markdown umwandeln und anzeigen
md_text = tree_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

|    | Variable            | Kategorie   | Zeichen                | Deutsch                        | Datentyp   | Einheit   | Beschreibung                        | Wert            |
|----|---------------------|-------------|------------------------|--------------------------------|------------|-----------|-------------------------------------|-----------------|
|  0 | baumart             | tree        | Baumart                | Baumart                        | string     | -         | Art des Baumes                      | Fagus silvatica |
|  1 | belaubung           | tree        | Belaubung              | Belaubung                      | string     | -         | Belaubungszustand zur Messung       | unbelaubt       |
|  2 | vitalitaet          | tree        | Vitalitaet             | Vitalität nach A. Roloff       | Int64      | Stufe     | Vitalitätsstufe nach Andreas Roloff | 1               |
|  3 | h_st_a              | tree        | $h_{\mathrm{StA}}$     | Höhe Stämmling A               | float64    | m         | Gesamthöhe des Stämmlings A         | 26.15           |
|  4 | h_st_b              | tree        | $h_{\mathrm{StB}}$     | Höhe Stämmling B               | float64    | m         | Gesamthöhe des Stämmlings B         | 27.20           |
|  5 | u_1m                | tree        | $u_{1\,\mathrm{m}}$    | Umfang auf 1~m                 | float64    | cm        | Stammumfang auf 1~m Höhe            | 140.00          |
|  6 | h_zwiesel           | tree        | $h_{\mathrm{Zwiesel}}$ | Höhe Zwiesel                   | float64    | m         | Höhe des Zwiesels über Boden        | 10.31           |
|  7 | h_ks                | tree        | $h_{\mathrm{KS}}$      | Höhe Kronensicherung           | float64    | m         | Höhe der KS-Anbringung über Boden   | 17.40           |
|  8 | l_ks                | tree        | $l_{\mathrm{KS}}$      | Länge Kronensicherung          | float64    | cm        | Länge der eingebauten KS            | 135.00          |
|  9 | u_st_a_h_ks         | tree        | $u_{\mathrm{StA,KS}}$  | Umfang Stämmling A auf Höhe KS | float64    | cm        | Stammumfang von A auf Höhe der KS   | 58.00           |
| 10 | u_st_b_h_ks         | tree        | $u_{\mathrm{StB,KS}}$  | Umfang Stämmling B auf Höhe KS | float64    | cm        | Stammumfang von B auf Höhe der KS   | 48.00           |
| 11 | standort_h_nn       | tree        | $h_{\mathrm{NN}}$      | Standort Höhe über n. N.       | float64    | m         | Standorthöhe über Normalnull        | 352.00          |
| 12 | standort_geo_breite | tree        | $\varphi$              | Standort Geo. Breite           | float64    | Grad      | Geografische Breite des Standorts   | 51.589476       |
| 13 | standort_geo_laenge | tree        | $\lambda$              | Standort Geo. Länge            | float64    | Grad      | Geografische Länge des Standorts    | 9.985242        |

### EXPORT: Daten exportieren für Weiterverarbeitung (.feather, .csv)

In [7]:
# DataFrame als Feather
tree_df.to_feather(data_export_directory / "tree.feather")
tree_df.to_csv(data_export_directory / "tree.csv", sep=";", index=False, encoding="utf-8")

### LATEX-EXPORT: Daten und Datendokumentation als Latex-Tabelle exportieren (.tex)

In [8]:
select_data_fields = ["Zeichen", "Deutsch", "Wert", "Datentyp", "Einheit"]
tree_df_latex = tree_df[select_data_fields].copy()

latex_string = tree_df_latex.to_latex(index=False, escape=False, column_format="llrrr",
                                      float_format="{:0.2f}".format)
caption = "Feldversuch 2 - Daten des Versuchsbaums"

save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_daten_des_versuchsbaums.tex


## Daten der Geräteanordnung am Baum (sensor_df)

### IMPORT: Daten und Datendokumentation

Die Datendokumentation ergibt sich aus `sensor_position_data_dict.json`.

Die `sensor_position.csv` enthält detaillierte Informationen zur Anordnung der Sensoren an den Bäumen im Rahmen des Experiments. Jede Zeile beschreibt die Platzierung eines Sensors.

In [9]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path/ "sensor_position_data_dict.json", "r", encoding="utf-8") as f:
    sensor_data_dict = json.load(f)

In [10]:
# Erzeuge dtype_dict dynamisch aus dem der Daten Dokumentation
dtype_dict = {key: value["Datentyp"] for key, value in sensor_data_dict.items()
    if value["Datentyp"] not in [None, ""]
}

# CSV einlesen mit dynamischen Datentypen
sensor_file = data_path / 'sensor_position.csv'
sensor_df = pd.read_csv(sensor_file, sep=';', decimal=',', na_values='NA',dtype=dtype_dict)

### ANALYSE: Explorative Datenanalyse

In [11]:
# In DataFrame umwandeln
sensor_data_dict_df = build_data_dict_df(sensor_data_dict)

# In Markdown umwandeln und anzeigen
md_text = sensor_data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

|    | Variable      | Kategorie       | Zeichen       | Deutsch     | Datentyp   | Einheit   | Beschreibung                             |
|----|---------------|-----------------|---------------|-------------|------------|-----------|------------------------------------------|
|  0 | location      | sensor_position | location      | Position    | string     | -         | Position des Sensors am Stamm            |
|  1 | height        | sensor_position | $h$           | Höhe        | Float64    | m         | Höhe des Sensors am Stamm                |
|  2 | diameter      | sensor_position | $d$           | Durchmesser | Float64    | cm        | Durchmesser des Stammes                  |
|  3 | direction     | sensor_position | direction     | Richtung    | string     | -         | Zug- oder Druckseite                     |
|  4 | type          | sensor_position | type          | Typ         | category   | -         | Typ des Sensors                          |
|  5 | sensor_id     | sensor_position | sensor\_id    | Sensor ID   | category   | -         | ID des Sensors                           |
|  6 | circumference | sensor_position | $u$           | Umfang      | Float64    | m         | Umfang des Stammes am Sensor             |
|  7 | position_id   | sensor_position | $position_\id | Position ID | Int64      | -         | Eindeutige ID für Sensorposition am Baum |
|  8 | note          | sensor_position | note          | Anmerkung   | string     | -         | Anmerkung zur Sensorposition             |

In [12]:
sensor_df["height"] = sensor_df["height"] / 100
sensor_df["circumference"] = sensor_df["circumference"] / 100
sensor_df["diameter"] = sensor_df["circumference"] / np.pi

sensor_df.head()

,position_id,type,sensor_id,location,height,circumference,direction,note,diameter
0,1,LS3,14:BF:E6,rope,18.3,<NA>,<NA>,<NA>,<NA>
1,2,LS3,14:99:1E,cable,17.4,<NA>,<NA>,<NA>,<NA>
2,3,TMS1,015,StA,18.0,0.45,west,<NA>,0.143239
3,4,TMS1,014,StB,18.0,0.4,west,<NA>,0.127324
4,5,TMS1,013,StA,15.0,0.67,west,<NA>,0.213268


### EXPORT: Daten exportieren für Weiterverarbeitung (.feather, .csv, .json)

In [13]:
# DataFrame als Feather
sensor_df.to_feather(data_export_directory / "sensor.feather")
sensor_df.to_csv(data_export_directory / "sensor.csv", sep=";", index=False, encoding="utf-8")

# Dict als JSON (UTF-8, sauber eingerückt)
with open(data_export_directory / "sensor_data_dict.json", "w", encoding="utf-8") as f:
    json.dump(sensor_data_dict, f,  indent=4, ensure_ascii=False)

### LATEX-EXPORT: Daten und Datendokumentation als Latex-Tabelle exportieren (.tex)

In [14]:
sensor_select_variables = ["type", "sensor_id", "location", "direction", "height", "circumference", "diameter"]

In [15]:
sensor_df_latex = sensor_df.copy()[sensor_select_variables]

# Ersetze Spaltennamen durch Formelzeichen
sensor_df_latex.columns = [sensor_data_dict[col]["Zeichen"] for col in sensor_select_variables]

latex_string = sensor_df_latex.to_latex(index=False, escape=False, column_format="lllrrrr",
    float_format="{:0.2f}".format)

caption = "Feldversuch 2 - Geräteanordnung"
save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_gerateanordnung.tex


In [16]:
sensor_data_dict_df = build_data_dict_df(sensor_data_dict, keys=sensor_select_variables, escape_index=True, select_latex_fields=True)
latex_string = sensor_data_dict_df.to_latex(index=False, escape=False)
caption = "Feldversuch 2 - Geräteanordnung Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_gerateanordnung_daten_dokumentation.tex


## Daten des Versuchsablaufs (series_df)

### IMPORT: Daten und Datendokumentation

Die Datendokumentation ergibt sich aus `series_data_dict.json`.

Die `series.csv` enthält das Protokoll des Versuchsablaufes der einzelnen Messungen. Jede Zeile repräsentiert eine einzelne Messung. 

In [17]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path/ "series_data_dict.json", "r", encoding="utf-8") as f:
    series_data_dict = json.load(f)

In [18]:
# Erzeuge dtype_dict dynamisch aus dem der Daten Dokumentation
dtype_dict = {key: value["Datentyp"] for key, value in series_data_dict.items()
    if value["Datentyp"] not in [None, ""]
}

# CSV einlesen mit dynamischen Datentypen
series_file = data_path / 'series.csv'
series_df = pd.read_csv(series_file, sep=';', decimal=',', na_values='NA', dtype=dtype_dict)

# Zeitspalte in Uhrzeit umwandeln
series_df["time"] = pd.to_datetime(series_df["time"], format="%H:%M:%S").dt.time

In [19]:
# Behandlungskategorien sortiert definieren
treatment_order = ['free', 'gefa_dynamic', 'cobra_static', 'cobra_static_slack']
treatment_category = CategoricalDtype(categories=treatment_order, ordered=True)
series_df["treatment"] = series_df["treatment"].astype(treatment_category)

### ANALYSE: Explorative Datenanalyse

Behandlungsvariante/Kronensicherung Kategorien:

- **`free`**: Der Baum konnte frei ohne Kronensicherung nach dem Release ausschwingen.
  
- **`gefa_dynamic`**: In ca. 2/3 der Baumhöhe wurde ein dynamisches Gefa Gurtband 4t dynamisch nach ZTV-Baumpflege mit leichtem Durchhang installiert. Das Ausschwingen wurde durch die KS abgedämpft. Da die Sicherung ohne Vorspannung installiert wurde, zeigen die Plots (/ls3/plots/force_vs_time_1/) von '14:99:1E' sowohl am Anfang als auch am Ende ca. 0 kN Kraft an (LogNr 1 bis 9).

- **`cobra_static`**: In ca. 2/3 der Baumhöhe wurde eine statische Cobra ultrastatic 7t (Dyneema) Sicherung installiert. Die Vorspannung betrug ca. 0,4 kN, wie sich in den Plots (/ls3/plots/force_vs_time_1/) von '14:99:1E' gut erkennen lässt. Durch das Zusammenziehen der Stämmlinge ist die Kronensicherung vor dem Release vollständig lastfrei (0 kN). Nach dem Release pendelt sich die Kraft ca. bei 0,4 kN ein (LogNr 10-18).

- **`cobra_static_slack`**: Ähnlich der `cobra_static`, jedoch wurde die Vorspannung entfernt. Aufgrund von Regen wurde nur eine Messung durchgeführt und die Serie frühzeitig abgebrochen.


In [20]:
# In DataFrame umwandeln
series_data_dict_df = build_data_dict_df(series_data_dict)

# In Markdown umwandeln und anzeigen
md_text = series_data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

|    | Variable             | Kategorie   | Zeichen                               | Deutsch            | Datentyp   | Einheit   | Beschreibung                                                              |
|----|----------------------|-------------|---------------------------------------|--------------------|------------|-----------|---------------------------------------------------------------------------|
|  0 | treatment            | series      | treatment                             | Behandlung         | category   | -         | Art der KS: \texttt{free}, \texttt{gefa\_dynamic}, \texttt{cobra\_static} |
|  1 | release_force_target | series      | $F_{\mathrm{release,target}}$         | Vorspannkraft-Soll | float64    | kN        | Geplante Vorspannkraft im Zugseil bei Release                             |
|  2 | id                   | series      | id                                    | ID Messung         | Int64      | -         | ID der Messung / Beobachtung                                              |
|  3 | time                 | series      | $t$                                   | Zeit               | string     | Zeit      | ca. Uhrzeit Beginn                                                        |
|  4 | 14:BF:E6             | series      | $\mathrm{LogNr\ LS3}_{\mathrm{Seil}}$ | LogNr. LS3 Zugseil | Int64      | -         | LogNr. LineScale3 im Zugseil                                              |
|  5 | 14:99:1E             | series      | $\mathrm{LogNr\ LS3}_{\mathrm{KS}}$   | LogNr. LS3 KS      | Int64      | -         | LogNr. LineScale3 in KS                                                   |

In [21]:
series_df.head()

,id,time,release_force_target,treatment,14:BF:E6,14:99:1E
0,1,10:10:00,2.5,free,1,<NA>
1,2,10:20:00,2.8,free,2,<NA>
2,3,10:28:00,2.8,free,3,<NA>
3,4,10:47:00,2.8,free,4,<NA>
4,5,10:53:00,2.4,free,5,<NA>


### EXPORT: Daten exportieren für Weiterverarbeitung (.feather, .csv, .json)

In [22]:
# DataFrame als Feather
series_df.to_feather(data_export_directory / "series.feather")
series_df.to_csv(data_export_directory / "series.csv", sep=";", index=False, encoding="utf-8")

# Dict als JSON (UTF-8, sauber eingerückt)
with open(data_export_directory / "series_data_dict.json", "w", encoding="utf-8") as f:
    json.dump(series_data_dict, f,  indent=4, ensure_ascii=False)

### LATEX-EXPORT: Daten und Datendokumentation als Latex-Tabelle exportieren (.tex)

In [23]:
series_select_variables = ["id", "time", "release_force_target", "treatment", "14:BF:E6", "14:99:1E"]

In [24]:
series_df_latex = series_df.copy()[series_select_variables]

# Ersetze Spaltennamen durch Formelzeichen
series_df_latex.columns = [series_data_dict[col]["Zeichen"] for col in series_select_variables]

series_df_latex["treatment"] = series_df_latex["treatment"].apply(
    lambda x: "\\texttt{" + str(x).replace("_", "\\_") + "}"
)

latex_string = series_df_latex.to_latex(index=False, escape=False, column_format="llrlrr",
                                        float_format="{:0.2f}".format)

caption = "Feldversuch 2 - Versuchsablauf"
caption_long = "Feldversuch 2 - Versuchsablauf"

save_latex_table(latex_string, caption, latex_export_directory, caption_long=caption_long)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_versuchsablauf.tex


In [25]:
series_data_dict_df = build_data_dict_df(series_data_dict, keys=series_select_variables, escape_index=True, select_latex_fields=True)

latex_string = series_data_dict_df.to_latex(index=False, escape=False)
caption = "Feldversuch 2 - Versuchsablauf Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_versuchsablauf_daten_dokumentation.tex
